In [36]:
import os

In [37]:
os.chdir("D:\Projects\PoultryDiagnose")

In [38]:
%pwd

'D:\\Projects\\PoultryDiagnose'

In [39]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [40]:
from PoultryDiseaseClassificationCNN.constants import *
from PoultryDiseaseClassificationCNN.utils.common import read_yaml, create_directories

In [41]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    
      

In [42]:
import os
import urllib.request as request
import zipfile
from PoultryDiseaseClassificationCNN import logger
from PoultryDiseaseClassificationCNN.utils.common import get_size

In [43]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [45]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-01 18:07:54,455: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-01 18:07:54,456: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-01 18:07:54,457: INFO: common: created directory at: artifacts]
[2024-05-01 18:07:54,458: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-01 18:07:56,652: INFO: 1170291011: artifacts/data_ingestion/PoultryDisease.zip download! with following info: 
Connection: close
Content-Length: 39628020
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "26d14deae5829a2c12c7a28d550e9ba362578667faa00668d4454c3ab32e4a23"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 402C:37EEC6:A63630:B90851:6632BD3D
Accept-Ranges: bytes
Date: Wed, 01 May 2024 22:07:58 GMT
Via: 1.1 varnish
X-Served-By: cache-yyz4581-YYZ
